In [2]:
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf

In [3]:
XLINES = np.arange(4068,4718,10)


def load_label(xlineidx):
    label = np.load('MODEL/LABEL/label_XL_%s.npy' % int(XLINES[xlineidx]))

    return label


def plot_overlay(image, label):
    (tmax, xmax) = np.shape(image)

    label_rgb = np.zeros((tmax, xmax, 4), 'uint8')
    label_rgb[:, :, 0] = 255
    label_rgb[:, :, 1] = 255 - 255 * label
    label_rgb[:, :, 2] = 255 - 255 * label
    label_rgb[:, :, 3] = 255 * label

    img = Image.fromarray(label_rgb, mode='RGBA')

    plt.imshow(image, cmap='gray')
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])


def patchify(data, label, size, number, threshold):
    (t_max, x_max) = label.shape
    X = np.zeros((number, size, size, 1))
    Y = np.zeros((number, size, size, 1))

    n = 0
    while n < number:
        x = random.randint(size // 2, x_max - size // 2)
        t = random.randint(size // 2, t_max - size // 2)
        if np.count_nonzero(label[t - size // 2:t + size // 2, x - size // 2:x + size // 2]) > threshold:
            X[n, :, :, 0] = data[t - size // 2:t + size // 2, x - size // 2:x + size // 2]
            Y[n, :, :, 0] = label[t - size // 2:t + size // 2, x - size // 2:x + size // 2]
            n = n + 1
    return X, Y

In [4]:
# Parameters
xlineidxs = np.arange(0,len(XLINES)+1,10)

size = 64
threshold = 0
num_train = 2000*len(xlineidxs)
num_val = 2000

# Validation data from inline 300
X_val = np.zeros((num_val, size, size, 1))
Y_val = np.zeros((num_val, size, size, 1))

xlineidx = 2
label = load_label(xlineidx)
seismic = np.load('./MODEL/TRAIN/trainer_XL_%s.npy' % int(XLINES[xlineidx]))

X_val, Y_val = patchify(seismic, label, size, num_val, threshold)

# Training data from inlines 100, 200, 400, 500
X_train = np.zeros((num_train, size, size, 1))
Y_train = np.zeros((num_train, size, size, 1))

In [5]:
n = 0
for xlineidx in xlineidxs:

    label = np.load('./MODEL/LABEL/label_XL_%s.npy' % int(XLINES[xlineidx]))
    seismic = np.load('./MODEL/TRAIN/trainer_XL_%s.npy' % int(XLINES[xlineidx]))

    X_train[n:n + num_train // len(xlineidxs), ...], Y_train[n:n + num_train // len(xlineidxs), ...] = patchify(seismic, label, size,
                                                                                        num_train // len(xlineidxs), threshold)
    n += num_train // len(xlineidxs)


def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = tf.keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p


def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = tf.keras.layers.UpSampling2D((2, 2))(x)
    concat = tf.keras.layers.Concatenate()([us, skip])
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c


def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c


def UNet():
    f = [16, 32, 64, 128, 256]
    inputs = tf.keras.layers.Input((size, size, 1))

    p0 = inputs
    c1, p1 = down_block(p0, f[0])  # 128 -> 64
    c2, p2 = down_block(p1, f[1])  # 64 -> 32
    c3, p3 = down_block(p2, f[2])  # 32 -> 16
    c4, p4 = down_block(p3, f[3])  # 16->8

    bn = bottleneck(p4, f[4])

    u1 = up_block(bn, c4, f[3])  # 8 -> 16
    u2 = up_block(u1, c3, f[2])  # 16 -> 32
    u3 = up_block(u2, c2, f[1])  # 32 -> 64
    u4 = up_block(u3, c1, f[0])  # 64 -> 128

    outputs = tf.keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = tf.keras.models.Model(inputs, outputs)
    return model

model = UNet()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])


history = model.fit(X_train,
                    Y_train,
                    validation_data=(X_val, Y_val),
                    epochs=50, verbose=1)


# serialize model to JSON
model_json = model.to_json()
with open("./MODEL/Fault_Detection_ModelV2.json", "w") as json_file:
    json_file.write(model_json)


# serialize weights to HDF5
model.save_weights("./MODEL/Fault_Detection_WeightV2.h5")

In [ ]:
plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()